In [16]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [17]:
#Get S&P 500 stocks list
stocks = pd.read_csv('sp_500_stocks.csv')
#stocks = stocks.drop([148,473])
print(stocks[148:150])
print(stocks[472:476])

    Symbol                  Name                  Sector
148  DISCK  Discovery (Series C)  Communication Services
149   DISH          Dish Network  Communication Services
    Symbol                  Name                  Sector
472   VTRS               Viatris             Health Care
473      V                  Visa  Information Technology
474    VNO  Vornado Realty Trust             Real Estate
475    VMC      Vulcan Materials               Materials


In [18]:
#check api call retrieves data

symbol='AAPL'
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url)
print(data)

data = requests.get(api_url).json()
print(data['symbol'])

<Response [200]>
AAPL


In [19]:
my_columns = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy


In [20]:
new_row = pd.Series(['AAPL', data['latestPrice'], data['marketCap'],  'N/A'],  index = my_columns)
final_dataframe = pd.concat([final_dataframe, new_row.to_frame().T], ignore_index=True)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,AAPL,151.94,2444666159800,N/A


In [ ]:
#feed data from api to required columns

final_dataframe = pd.DataFrame(columns = my_columns)
for i,symbol in enumerate(stocks['Symbol']):
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    new_row = pd.Series([symbol, data['latestPrice'], data['marketCap'],  'N/A'],  index = my_columns)
    final_dataframe = pd.concat([final_dataframe, new_row.to_frame().T], ignore_index=True)

final_dataframe

In [21]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [26]:
#flood data in batch api calls for faster retreival

symbol_groups = list(chunks(stocks['Symbol'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        new_row = pd.Series([symbol, data[symbol]['quote']['latestPrice'], data[symbol]['quote']['marketCap'],  'N/A'],  index = my_columns)
        final_dataframe = pd.concat([final_dataframe, new_row.to_frame().T], ignore_index=True)
print(len(final_dataframe))
final_dataframe

502


,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,MMM,124.01,70025969155,N/A
1,AOS,52.6,6912442012,N/A
2,ABT,102.491,180442647573,N/A
3,ABBV,157.32,269454937112,N/A
4,ABMD,282.16,12831830385,N/A
...,...,...,...,...
497,YUM,114.004,32756794374,N/A
498,ZBRA,275.17,14841956573,N/A
499,ZBH,117.29,23935920665,N/A
500,ZION,52.65,7683246014,N/A


In [27]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:30000000


In [28]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = position_size // final_dataframe['Price'][i]
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,MMM,124.01,70025969155,481.0
1,AOS,52.6,6912442012,1136.0
2,ABT,102.491,180442647573,583.0
3,ABBV,157.32,269454937112,379.0
4,ABMD,282.16,12831830385,211.0
...,...,...,...,...
497,YUM,114.004,32756794374,524.0
498,ZBRA,275.17,14841956573,217.0
499,ZBH,117.29,23935920665,509.0
500,ZION,52.65,7683246014,1135.0


In [29]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [30]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [31]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)
    
writer.save()

/var/folders/sy/r631gy8n4r92m2pzw9tynp8w0000gn/T/ipykernel_94942/374261448.py:12: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
